In [9]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
import joblib
import mysql.connector
from datetime import datetime
import os

# Load the SVM model and encoder
with open('svm_model_with_encoder.pkl', 'rb') as f:
    data = joblib.load(f)
    model = data['model']
    encoder = data['encoder']

# Initialize FaceNet and MTCNN
embedder = FaceNet()
detector = MTCNN()

# Test database connection
try:
    db = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="@Sriram806",
        database="attendence"
    )
    cursor = db.cursor()
    print("Database connected successfully.")
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit()

# Create directories for saving outputs
frames_dir = "frames"
faces_dir = "faces"
os.makedirs(frames_dir, exist_ok=True)
os.makedirs(faces_dir, exist_ok=True)

# Attendance tracker (Keeps track of detected students)
attendance_tracker = {}

# Function to get face embedding
def get_embedding(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

# Function to log attendance in the database
def log_attendance(student_id, timestamp, status, face_image_path):
    try:
        cursor.execute(
            "INSERT INTO attendance (student_id, timestamp, status, face_image_path) VALUES (%s, %s, %s, %s)",
            (student_id, timestamp, status, face_image_path)
        )
        db.commit()
        print(f"Attendance logged for {student_id}: {status}")
    except mysql.connector.Error as e:
        print(f"Error logging attendance: {e}")

# Save frame or face images
def save_image(directory, filename, image):
    filepath = os.path.join(directory, filename)
    cv2.imwrite(filepath, image)
    return filepath

# Capture video from webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the camera. Check your device and permissions.")
    exit()

# Parameters for processing
start_time = datetime.now()
frame_interval = 10  # Seconds between frames

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Detect faces and process them
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb_frame)

    for face in faces:
        x, y, w, h = face['box']
        x, y = abs(x), abs(y)
        face_img = rgb_frame[y:y+h, x:x+w]

        try:
            face_img_resized = cv2.resize(face_img, (160, 160))
            embedding = get_embedding(face_img_resized)
            embedding = np.expand_dims(embedding, axis=0)

            prediction = model.predict(embedding)
            class_label = encoder.inverse_transform(prediction)[0]

            # Track attendance for each student
            if class_label not in attendance_tracker:
                attendance_tracker[class_label] = 0
            attendance_tracker[class_label] += 1

            # Save detected face
            face_filename = f"{class_label}_frame_{frame_count + 1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            face_path = save_image(faces_dir, face_filename, cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))

            # Log attendance only once per student
            if attendance_tracker[class_label] == 1:
                log_attendance(class_label, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Present", face_path)

        except Exception as e:
            print(f"Error processing face: {e}")

    frame_count += 1

    # Display the frame
    cv2.imshow('Face Recognition Attendance', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Final attendance summary
print("\nAttendance Summary:")
for student_name, count in attendance_tracker.items():
    print(f"{student_name}: {'Present' if count >= 1 else 'Absent'}")

# Close database connection
try:
    db.close()
    print("Database connection closed.")
except mysql.connector.Error as e:
    print(f"Error closing the database connection: {e}")


Database connected successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Error logging attendance: 1366 (HY000): Incorrect integer value: 'Yeswanth' for column 'student_id' at row 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━

import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
import joblib
import mysql.connector
from datetime import datetime
import os

# Load the SVM model and encoder
with open('svm_model_with_encoder.pkl', 'rb') as f:
    data = joblib.load(f)
    model = data['model']
    encoder = data['encoder']

# Initialize FaceNet and MTCNN
embedder = FaceNet()
detector = MTCNN()

# Test database connection
try:
    db = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="@Sriram806",
        database="attendence"
    )
    cursor = db.cursor()
    print("Database connected successfully.")
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit()

# Create directories for saving outputs
frames_dir = "frames"
faces_dir = "faces"
os.makedirs(frames_dir, exist_ok=True)
os.makedirs(faces_dir, exist_ok=True)

# Attendance tracker
attendance_tracker = {}

# Function to get face embedding
def get_embedding(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

# Function to log attendance in the database
def log_attendance(student_id, timestamp, status, face_image_path):
    try:
        cursor.execute(
            "INSERT INTO attendance (student_id, timestamp, status, face_image_path) VALUES (%s, %s, %s, %s)",
            (student_id, timestamp, status, face_image_path)
        )
        db.commit()
        print(f"Attendance logged for {student_id}: {status}")
    except mysql.connector.Error as e:
        print(f"Error logging attendance: {e}")

# Save frame or face images
def save_image(directory, filename, image):
    filepath = os.path.join(directory, filename)
    cv2.imwrite(filepath, image)
    return filepath

# Capture video from webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the camera. Check your device and permissions.")
    exit()

# Parameters for processing
start_time = datetime.now()
frame_interval = 10  # Seconds between frames
total_frames = 6
frame_count = 0

while frame_count < total_frames:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    elapsed_time = (datetime.now() - start_time).seconds
    if elapsed_time >= frame_count * frame_interval:
        print(f"Processing frame {frame_count + 1}")
        frame_filename = f"frame_{frame_count + 1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        save_image(frames_dir, frame_filename, frame)

        # Detect faces
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(rgb_frame)

        for face in faces:
            x, y, w, h = face['box']
            x, y = abs(x), abs(y)
            face_img = rgb_frame[y:y+h, x:x+w]

            try:
                face_img_resized = cv2.resize(face_img, (160, 160))
                embedding = get_embedding(face_img_resized)
                embedding = np.expand_dims(embedding, axis=0)

                prediction = model.predict(embedding)
                class_label = encoder.inverse_transform(prediction)[0]

                # Track attendance
                if class_label not in attendance_tracker:
                    attendance_tracker[class_label] = 0
                attendance_tracker[class_label] += 1

                # Save detected face
                face_filename = f"{class_label}_frame_{frame_count + 1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
                face_path = save_image(faces_dir, face_filename, cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))

                # Log attendance
                if attendance_tracker[class_label] == 1:
                    log_attendance(class_label, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Present", face_path)

            except Exception as e:
                print(f"Error processing face: {e}")

        frame_count += 1

# Release resources
cap.release()

# Final attendance summary
print("\nAttendance Summary:")
for student_name, count in attendance_tracker.items():
    print(f"{student_name}: {'Present' if count >= 4 else 'Absent'}")

# Close database connection
db.close()

In [8]:
import matplotlib.pyplot as plt
import cv2

# Load a saved frame or face
image = cv2.imread("frames/frame_1_20231118_123456.jpg")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
